In [113]:
import numpy as np
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../scripts')) 
from sklearn.model_selection import train_test_split
from LinearRegressionGradientDescent import CustomLinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_csv("../data/processed/cleaned_weather_data.csv")

from sklearn.metrics import r2_score

df.drop(['datetime'], axis=1, inplace=True)
df.drop(['visibility'], axis=1, inplace=True)

X = df.drop(['target_tempmin'], axis=1)
Y = df['target_tempmin']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# maxR2_score=0
# maxRS=0
# for i in range(1,500):
#     X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=i, test_size=.20)
#     lin_reg=CustomLinearRegression()
#     lin_reg.fit(X_train,Y_train)
#     y_pred=lin_reg.predict(X_test)
#     if np.isnan(y_pred).any():
#         continue
#     R2=r2_score(Y_test,y_pred)
#     if R2>maxR2_score:
#         maxR2_score=R2
#         maxRS=i
# print('Best R2 Score is', maxR2_score ,'on Random_state', maxRS)
maxRS = 204

In [114]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=maxRS, test_size=.20)
lin_reg=CustomLinearRegression()
lin_reg.fit(X_train,Y_train)
y_pred=lin_reg.predict(X_test)
print('\033[1m'+'Predicted Wins:'+'\033[0m\n',y_pred)
print('\n')
print('\033[1m'+'Actual Wins:'+'\033[0m\n',Y_test)

Predicted Wins:
 [19.90389513 17.3667135  23.80404061 ... 18.12693344 18.97553148
 11.3557516 ]


Actual Wins:
 659     20.1
7525    16.0
5864    23.0
5049    20.0
5275    12.1
        ... 
6895    22.5
6506    13.0
374     18.9
5719    21.0
670     12.1
Name: target_tempmin, Length: 1701, dtype: float64


In [115]:
print('\033[1m'+' Error :'+'\033[0m')
print('Mean absolute error (MAE):', mean_absolute_error(Y_test,y_pred))
print('Mean squared error (MSE):', mean_squared_error(Y_test,y_pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(Y_test,y_pred)))
print('\n')
from sklearn.metrics import r2_score
print('\033[1m'+' R2 Score :'+'\033[0m')
print(r2_score(Y_test,y_pred,multioutput='variance_weighted'))

 Error :
Mean absolute error (MAE): 1.047269983813197
Mean squared error (MSE): 1.8215468535853356
Root Mean Squared Error (RMSE): 1.3496469366413335


 R2 Score :
0.929593832823427


In [116]:
import joblib
joblib.dump(lin_reg,'../models/Next_Tmin_Forecast_Final.pkl')

['../models/Next_Tmin_Forecast_Final.pkl']

In [117]:
Model = joblib.load("../models/Next_Tmin_Forecast_Final.pkl")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Dự đoán trên tập kiểm tra
actual = np.array(Y_test)
predicted = np.array(Model.predict(X_test))

# Tạo DataFrame
df_Predicted = pd.DataFrame({
    "Actual Values": actual,
    "Predicted Values": predicted
}, index=range(len(actual)))

df_Predicted["Sai lệch"] = df_Predicted["Actual Values"] - df_Predicted["Predicted Values"]
# Đếm số trường có độ lệch lớn hơn 2 °C (dựa trên giá trị tuyệt đối)
large_deviation_count = len(df_Predicted[abs(df_Predicted["Sai lệch"]) > 2])
print(f"\nSố trường có độ lệch lớn hơn 2 °C (target_tempmax): {large_deviation_count}")

# Hiển thị toàn bộ DataFrame
print("\nDataFrame với giá trị thực tế và dự đoán trên thang gốc:")
print(df_Predicted)

# Đặt lại thiết lập hiển thị về mặc định sau khi in
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')


Số trường có độ lệch lớn hơn 2 °C (target_tempmax): 221

DataFrame với giá trị thực tế và dự đoán trên thang gốc:
      Actual Values  Predicted Values  Sai lệch
0              20.1         19.903895  0.196105
1              16.0         17.366714 -1.366714
2              23.0         23.804041 -0.804041
3              20.0         20.406656 -0.406656
4              12.1         12.503818 -0.403818
5              19.0         19.299169 -0.299169
6              28.0         25.832064  2.167936
7              13.1         15.972124 -2.872124
8              25.1         24.631954  0.468046
9              12.0         10.624401  1.375599
10             18.1         17.413919  0.686081
11             27.1         26.568991  0.531009
12             12.0         10.978967  1.021033
13             25.0         24.490356  0.509644
14             27.0         25.928883  1.071117
15             18.1         18.067919  0.032081
16             22.0         24.269366 -2.269366
17             22.0  

# <b> Đánh giá mô hình </b>

### 1. Tổng quan

Để đánh giá hiệu quả của mô hình dự đoán nhiệt độ nhỏ nhất, nhóm sử dụng các chỉ số phổ biến: `MAE`, `MSE`, `RMSE`, và `R² Score`. Ngoài ra, phân tích sai lệch giữa giá trị thực tế và dự đoán được thực hiện để đánh giá khả năng tổng quát của mô hình trong thực tiễn.

### 2. Chỉ số đánh giá mô hình

Sau khi huấn luyện và kiểm tra trên tập test (1701 mẫu), các chỉ số thu được như sau:

| **Chỉ số đánh giá**                | **Giá trị**     |
|------------------------------------|-----------------|
| MAE (Sai số tuyệt đối trung bình)  | 1.047°C         |
| MSE (Sai số bình phương trung bình)| 1.822°C²        |
| RMSE (Căn bậc hai của MSE)         | 1.350°C         |
| R² Score (Hệ số xác định)          | 0.930           |

##### 2.1. Mean Absolute Error - MAE
`MAE` đo sai số tuyệt đối trung bình giữa giá trị dự đoán và thực tế, được tính bằng công thức:

$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
$$

- $y_i$: giá trị thực tế
- $\hat{y}_i$: giá trị dự đoán

Với MAE ≈ 1.047°C, trung bình các dự đoán lệch khoảng 1.05°C so với giá trị thực tế.

- **Ví dụ từ DataFrame**:
  - Dòng 0: Thực tế = 20.1°C, dự đoán = 19.903895°C, sai số = 0.196105°C (rất nhỏ, mô hình dự đoán tốt).
  - Dòng 7: Thực tế = 13.1°C, dự đoán = 15.972124°C, sai số = -2.872124°C (sai số lớn).

**Nhận xét**:  
- MAE ≈ 1.047°C là kết quả tốt trong dự báo nhiệt độ nhỏ nhất, vì sai số khoảng 1°C thường được chấp nhận trong các ứng dụng dự báo thời tiết thông thường. Tuy nhiên, sai số lớn ở một số điểm (như dòng 7) cần được lưu ý để cải thiện.

##### 2.2. Mean Squared Error - MSE
`MSE` đo trung bình bình phương sai số, nhấn mạnh các sai số lớn, được tính bằng công thức:

$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

- $y_i$: giá trị thực tế
- $\hat{y}_i$: giá trị dự đoán

**Nhận xét**:  
- MSE ≈ 1.822°C² cho thấy mô hình có một số sai số lớn (ví dụ, dòng 7: sai số -2.872124°C, bình phương ≈ 8.25°C²), nhưng tổng thể sai số được kiểm soát tốt. Giá trị này phản ánh rằng mô hình hoạt động ổn định trong phần lớn trường hợp.

##### 2.3. Root Mean Squared Error - RMSE
`RMSE` là căn bậc hai của MSE, đưa sai số về đơn vị °C, phản ánh mức độ sai lệch trung bình với trọng số lớn hơn cho sai số lớn.

**Nhận xét**:  
- RMSE ≈ 1.350°C cao hơn MAE, cho thấy có các sai số lớn (như dòng 7: -2.872124°C hoặc dòng 1699: 2.024469°C) làm tăng giá trị. Kết quả này chấp nhận được trong dự báo nhiệt độ nhỏ nhất, nhưng không lý tưởng cho các ứng dụng yêu cầu độ chính xác cao (ví dụ, dự báo sương giá).

##### 2.4. R² Score
`R²` đo lường tỷ lệ phương sai của biến mục tiêu được giải thích bởi mô hình, được tính bằng công thức:

$$
R^2 = 1 - \frac{SS_{RES}}{SS_{TOT}} = 1 - \frac{\sum_i (y_i - \hat{y}_i)^2}{\sum_i (y_i - \bar{y})^2}
$$

- $y_i$: giá trị thực tế  
- $\hat{y}_i$: giá trị dự đoán  
- $\bar{y}$: giá trị trung bình  
- $SS_{RES}$: tổng bình phương sai số  
- $SS_{TOT}$: tổng phương sai thực tế  

**Nhận xét**:  
- R² ≈ 0.930 cho thấy mô hình giải thích được 93% sự biến thiên của nhiệt độ nhỏ nhất, là kết quả rất tốt. Chỉ 7% phương sai chưa được giải thích, có thể do các yếu tố thời tiết phức tạp hoặc thiếu đặc trưng. Điều này cho thấy tiềm năng ứng dụng cao của mô hình trong dự báo thời tiết ngắn hạn.

### 3. Phân tích sai số và độ lệch

- **Tổng số mẫu kiểm tra**: 1.701
- **Số mẫu có sai lệch lớn hơn 2°C**: 221 mẫu (~13.0%)
- **Tỷ lệ dự đoán chính xác (sai lệch ≤ 2°C)**: ~87.0%

**Nhận xét**:  
- Tỷ lệ 13.0% điểm có sai số >2°C (221/1.701) là đáng chú ý, cho thấy mô hình gặp khó khăn trong một số trường hợp. Tuy nhiên, 87.0% điểm có sai số ≤2°C là kết quả tích cực, phù hợp cho các ứng dụng dự báo thông thường.
- Ví dụ sai số lớn:
  - Dòng 7 (1998-01-23): Sai số = -2.872124°C (thực tế = 13.1°C, dự đoán = 15.972124°C). Dữ liệu cho thấy `cloudcover` = 1.055026 và `windspeed` = 1.407341. Mô hình có thể đã dự đoán nhiệt độ cao hơn do đánh giá quá cao tác động giữ nhiệt của độ che phủ mây, mà không tính đủ hiệu ứng làm mát của gió mạnh.
  - Dòng 1699 (2025-03-04): Sai số = 2.024469°C (thực tế = 21.0°C, dự đoán = 18.975531°C). Với `humidity` = 0.710641 và `cloudcover` = 0.942177, mô hình có thể đã đánh giá thấp mức độ giữ nhiệt ban đêm do mây dày và độ ẩm cao.

Các đặc trưng như `cloudcover`, `windspeed`, và `humidity` có vai trò lớn trong dự đoán nhiệt độ trung bình nhỏ nhất. Sai số lớn thường xảy ra khi các đặc trưng này có giá trị bất thường hoặc khi mô hình không nắm bắt được tương tác phi tuyến giữa chúng (ví dụ, mây dày kết hợp với gió mạnh làm giảm nhiệt độ, hoặc mây dày và độ ẩm cao giữ nhiệt mạnh).

### 4. Kết luận

Mô hình dự đoán nhiệt độ nhỏ nhất hoạt động hiệu quả trong phần lớn trường hợp, đặc biệt với các điều kiện thời tiết thông thường, nhờ khai thác tốt các đặc trưng như nhiệt độ trễ và độ che phủ mây. Tuy nhiên, mô hình gặp khó khăn trong các điều kiện thời tiết phức tạp, dẫn đến một số sai số lớn, đặc biệt khi các yếu tố như độ che phủ mây, tốc độ gió, và độ ẩm có tương tác phi tuyến. Để cải thiện, nên xem xét sử dụng các mô hình phi tuyến, bổ sung đặc trưng tương tác và phân tích sâu các điểm sai số lớn để nâng cao độ chính xác.